In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import obspy
import os
from obspy.taup import TauPyModel
from obspy.geodetics import kilometer2degrees
import copy
import matplotlib
from matplotlib.cm import get_cmap
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.basemap import Basemap
import shapefile
from fatiando import gridder, utils
import scipy.io
import matplotlib.cm as cm
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

# Coletando os dados das estações BP

In [ ]:
ev_list = []
ev_listS = []
#for root, dirs, files in os.walk('/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/redeBP-BB/transition_zone/syngine_filtered_PP/a_05/'):
for root, dirs, files in os.walk('/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/redeBP-BB/transition_zone/rede_BB/event_data_PP_selected/a_05/'):
    for datafile in files:
        if datafile.endswith('.eqr'):
            ev_list.append(os.path.join(root, datafile))
ev_listS = sorted(ev_list)

In [ ]:
ev = obspy.Stream()
for i,j in enumerate(ev_listS):
    ev += obspy.read(j)

In [ ]:
event_DD = []
event_MM = []
event_YYYY = []
event_hh = []
event_mm = []
event_julday = []
event_depth = []
event_lat = []
event_long = []
event_dist = []
event_gcarc = []
event_sta = []
event_channel = []
event_ray = []
sta_lat = []
sta_long = []
sta_channel = []
sta_data = []
sta_time = []
event_starttime = []
event_endtime = []


for i,j in enumerate(ev):
    if j.stats.sac.gcarc > 30:
            event_time = (j.stats.starttime)
            event_starttime.append(j.stats.starttime)
            event_endtime.append(j.stats.endtime)
            event_DD.append("{0:02.0f}".format(event_time.day))
            event_MM.append("{0:02.0f}".format(event_time.month))
            event_YYYY.append(event_time.year)
            event_hh.append("{0:02.0f}".format(event_time.hour))
            event_mm.append("{0:02.0f}".format(event_time.minute))
            event_julday.append(event_time.julday)
            event_depth.append(j.stats.sac.evdp)
            #event_depth.append(j.stats.sac.evdp/1000) #para os dados sintéticos
            event_lat.append(j.stats.sac.evla)
            event_long.append(j.stats.sac.evlo)
            event_dist.append(j.stats.sac.dist)
            event_gcarc.append(j.stats.sac.gcarc)
            event_sta.append(j.stats.station)
            event_ray.append(j.stats.sac.user8)
            sta_lat.append(j.stats.sac.stla)
            sta_long.append(j.stats.sac.stlo)
            sta_data.append(j.data[100:2700])
            sta_time.append(j.times()[100:2700]-10)

In [ ]:
fig=plt.figure(figsize=(20,10))


project_Lat = -5
project_Lon = -45

m = Basemap(resolution='l',projection='cyl',lat_0=project_Lat, lon_0=project_Lon,llcrnrlon=-170.,
            llcrnrlat=-85.,urcrnrlon=65.,urcrnrlat=85.)


sf = shapefile.Reader('/home/diogo/dados_doutorado/parnaiba_basin/SIG_dados/shapes/bacia_parnaiba/bacia_parnaiba.shp')
for shape in sf.shapes():
    # adding polygon(s)
    parts = list(shape.parts) + [len(shape.points)]
    partlims = zip(parts[:-1], parts[1:])
    for i1, i2 in partlims:
        points = shape.points[i1:i2]
        lon, lat = zip(*points)
        x,y = m(lon, lat)
        m.plot(x, y, '-', lw=0.75, color='k')


for lon, lat in zip(event_long,event_lat):
    x,y = m(lon, lat)
    msize = 10
    m.plot(x, y, '*',markersize=msize,markeredgecolor='k',markerfacecolor='y')

for lon, lat in zip(sta_long,sta_lat):
    x,y = m(lon, lat)
    msize = 10
    m.plot(x, y, '^',markersize=msize,markeredgecolor='k',markerfacecolor='grey')

for evlon_0, evlat_0, evlon_c,evlat_c in zip(sta_long,sta_lat,event_long,event_lat):
    m.drawgreatcircle(evlon_0, evlat_0, evlon_c,evlat_c,linewidth=2,color='grey',alpha=0.1)

    
m.fillcontinents(color='whitesmoke',lake_color=None)
m.drawcoastlines(color='k',zorder=10)
m.drawmeridians(np.arange(0, 360, 20),color='lightgrey')
m.drawparallels(np.arange(-90, 90, 10),color='lightgrey')

sf = shapefile.Reader('/home/diogo/dados_doutorado/parnaiba_basin/SIG_dados/shapes/bacia_parnaiba/bacia_parnaiba.shp')
for shape in sf.shapes():
    # adding polygon(s)
    parts = list(shape.parts) + [len(shape.points)]
    partlims = zip(parts[:-1], parts[1:])
    for i1, i2 in partlims:
        points = shape.points[i1:i2]
        x, y = zip(*points)
        m.plot(x, y, '-', lw=3, color='k')

plt.title('Events Selected Locataion')

## Criando um modelo de terra através do obspy.taup.taup_create.build_taup_model

In [ ]:
obspy.taup.taup_create.build_taup_model('/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/redeBP-BB/transition_zone/velocity_model/IASP91_velocity_model/iasp91_10.tvel',output_folder='/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/redeBP-BB/transition_zone/velocity_model/IASP91_velocity_model/')

In [ ]:
model_10_km = TauPyModel(model='/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/redeBP-BB/transition_zone/velocity_model/IASP91_velocity_model/iasp91_10.npz')

dtype=[('top_depth', '<f8'), ('bot_depth', '<f8'), ('top_p_velocity', '<f8'), ('bot_p_velocity', '<f8'), ('top_s_velocity', '<f8'), ('bot_s_velocity', '<f8'), ('top_density', '<f8'), ('bot_density', '<f8'), ('top_qp', '<f8'), ('bot_qp', '<f8'), ('top_qs', '<f8'), ('bot_qs', '<f8')])

In [ ]:
top_depth = []
bot_depth = []
top_p_velocity = []
bot_p_velocity = []
top_s_velocity = []
bot_s_velocity = []
top_density = []
bot_density = []
top_qp = []
bot_qp = []
top_qs = []
bot_qs = []
for i,j in enumerate(model_10_km.model.s_mod.v_mod.layers):
    top_depth.append(j[0])
    bot_depth.append(j[1])
    top_p_velocity.append(j[2])
    bot_p_velocity.append(j[3])
    top_s_velocity.append(j[4])
    bot_s_velocity.append(j[5])
    top_density.append(j[6])
    bot_density.append(j[7])
    top_qp.append(j[8])
    bot_qp.append(j[9])
    top_qs.append(j[10])
    bot_qs.append(j[11])

In [ ]:
plt.figure(figsize=(5,10))
plt.title('IASP91 Earth Model 10 km thick')
plt.plot(top_p_velocity,top_depth,'b',label='P Velocity km/s')
plt.plot(top_s_velocity,top_depth,'g',label='S Velocity km/s')
plt.plot(top_density,top_depth,'r',label='Density')
plt.legend()
plt.ylim(6400,0)
plt.xlim(-0.5,14)

In [ ]:
radius = model_10_km.model.radius_of_planet
camadas = model_10_km.model.s_mod.v_mod.get_discontinuity_depths()

In [ ]:
print(camadas)

# Calculando os caminhos dos raios para os eventos das estações BP

# Onda P

In [ ]:
arrivalsP = []
for i,j in enumerate(event_depth):
    arrivalsP.append(model_10_km.get_travel_times_geo(
                                    source_depth_in_km=j, source_latitude_in_deg=event_lat[i], 
                                    source_longitude_in_deg=event_long[i], receiver_latitude_in_deg=sta_lat[i], 
                                    receiver_longitude_in_deg=sta_long[i], phase_list='P'))

# Onda Ps de 300 a 800

In [ ]:
phase_lst = ['P'+str(i)+'s' for i in range(300,810,10)]

In [ ]:
print(phase_lst)

In [ ]:
arrivals = []
for i,j in enumerate(event_depth):
    arrivals.append(model_10_km.get_travel_times_geo(
                                    source_depth_in_km=j, source_latitude_in_deg=event_lat[i], 
                                    source_longitude_in_deg=event_long[i], receiver_latitude_in_deg=sta_lat[i], 
                                    receiver_longitude_in_deg=sta_long[i], phase_list=phase_lst))

In [ ]:
dist_event = [[]]*len(arrivals)
depth = [[]]*len(arrivals)
time = [[]]*len(arrivals)

for i,j in enumerate(arrivals):
    for k,l in enumerate(j):
            time[i].append(l.time - arrivalsP[i][0].time)
            depth[i].append(float(l.name[1:-1]))
            dist_event[i].append(l.distance)

### Salvando os arquivos TXT com os tempos, lat, long, e distância:

In [ ]:
dist_event_txt = open('/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/redeBP-BB/transition_zone/velocity_model/Phases/dist_event/P_dist_event.txt', 'w')

for i,j in enumerate(dist_event):
    dist_event_txt.write(str(list(j))+'\n')
dist_event_txt.close()

In [ ]:
depth_txt = open('/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/redeBP-BB/transition_zone/velocity_model/Phases/depth/P_depth.txt', 'w')

for i,j in enumerate(depth):
    depth_txt.write(str(list(j))+'\n')
depth_txt.close()

In [ ]:
time_txt = open('/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/redeBP-BB/transition_zone/velocity_model/Phases/time/P_time.txt', 'w')

for i,j in enumerate(time):
    time_txt.write(str(list(j))+'\n')
time_txt.close()

# Onda PpPs de 300 a 800

In [ ]:
phase_lst_PpPs = ['PPv'+str(i)+'s' for i in range(300,810,10)]

In [ ]:
print(phase_lst_PpPs)

In [ ]:
arrivals_PpPs = []
for i,j in enumerate(event_depth):
    arrivals_PpPs.append(model_10_km.get_pierce_points_geo(
                                    source_depth_in_km=j, source_latitude_in_deg=event_lat[i], 
                                    source_longitude_in_deg=event_long[i], receiver_latitude_in_deg=sta_lat[i], 
                                    receiver_longitude_in_deg=sta_long[i], phase_list=phase_lst_PpPs))

In [ ]:
dist_event_PpPs = [[]]*len(arrivals_PpPs)
depth_PpPs = [[]]*len(arrivals_PpPs)
time_PpPs = [[]]*len(arrivals_PpPs)

for i,j in enumerate(arrivals_PpPs):
    for k,l in enumerate(j):
            time_PpPs[i].append(l.time - arrivalsP[i][0].time)
            depth_PpPs[i].append(float(l.name[-4:-1]))
            dist_event_PpPs[i].append(l.distance)

### Salvando os arquivos TXT com os tempos, lat, long, e distância:

In [ ]:
dist_event_txt_PpPs = open('/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/redeBP-BB/transition_zone/velocity_model/Phases/dist_event/PpPs_dist_event.txt', 'w')

for i,j in enumerate(dist_event_PpPs):
    dist_event_txt_PpPs.write(str(list(j))+'\n')
dist_event_txt_PpPs.close()

In [ ]:
depth_txt_PpPs = open('/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/redeBP-BB/transition_zone/velocity_model/Phases/depth/PpPs_depth.txt', 'w')

for i,j in enumerate(depth_PpPs):
    depth_txt_PpPs.write(str(list(j))+'\n')
depth_txt_PpPs.close()

In [ ]:
time_txt_PpPs = open('/home/diogo/dados_doutorado/parnaiba_basin/RF_migration/redeBP-BB/transition_zone/velocity_model/Phases/time/PpPs_time.txt', 'w')

for i,j in enumerate(time_PpPs):
    time_txt_PpPs.write(str(list(j))+'\n')
time_txt_PpPs.close()